# Защита персональных данных клиентов

## Задача

Вам нужно защитить данные клиентов страховой компании «Хоть потоп». Разработайте такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обоснуйте корректность его работы.

Нужно защитить данные, чтобы при преобразовании качество моделей машинного обучения не ухудшилось. Подбирать наилучшую модель не требуется.

## Навигация (структура в VS code)


-   Задача
-   Предподготовка данных
-   Преоразование матрицы признаков для защиты персональных данных
-   Какие действия необходимо сделать
-   Создание рандомного множителя и рандомного смещения
-   Разбиение на тренировочную и тестовую выборку
-   Преобразуем матрицу признаков
-   Проверка моделей не преобразованной матрицы признаков и с шифтом
-   Вывод


## Описание данных

##### Признаки:

- Пол
- Возраст
- Зарплата
- Члены семьи (количество)

##### Таргет

- Количество страховых выплат за последние 5 лет



In [14]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

RANDOM_STATE = 42

In [15]:
data = pd.read_csv('insurance.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Пол                5000 non-null   int64  
 1   Возраст            5000 non-null   float64
 2   Зарплата           5000 non-null   float64
 3   Члены семьи        5000 non-null   int64  
 4   Страховые выплаты  5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


# Предподготовка данных

In [16]:
data.columns = ['sex', 'age', 'salary', 'family_count', 'insurance']
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sex           5000 non-null   int64  
 1   age           5000 non-null   float64
 2   salary        5000 non-null   float64
 3   family_count  5000 non-null   int64  
 4   insurance     5000 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 195.4 KB


In [17]:
data.describe().round(1)

,sex,age,salary,family_count,insurance
count,5000.0,5000.0,5000.0,5000.0,5000.0
mean,0.5,31.0,39916.4,1.2,0.1
std,0.5,8.4,9900.1,1.1,0.5
min,0.0,18.0,5300.0,0.0,0.0
25%,0.0,24.0,33300.0,0.0,0.0
50%,0.0,30.0,40200.0,1.0,0.0
75%,1.0,37.0,46600.0,2.0,0.0
max,1.0,65.0,79000.0,6.0,5.0


In [18]:
data.duplicated().sum()

153

In [19]:
data = data.drop_duplicates()
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4847 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sex           4847 non-null   int64  
 1   age           4847 non-null   float64
 2   salary        4847 non-null   float64
 3   family_count  4847 non-null   int64  
 4   insurance     4847 non-null   int64  
dtypes: float64(2), int64(3)
memory usage: 227.2 KB


In [20]:
data['insurance'].unique()

array([0, 1, 2, 3, 5, 4], dtype=int64)

In [21]:
data.describe().round(1)

,sex,age,salary,family_count,insurance
count,4847.0,4847.0,4847.0,4847.0,4847.0
mean,0.5,31.0,39895.8,1.2,0.2
std,0.5,8.5,9973.0,1.1,0.5
min,0.0,18.0,5300.0,0.0,0.0
25%,0.0,24.0,33200.0,0.0,0.0
50%,0.0,30.0,40200.0,1.0,0.0
75%,1.0,37.0,46600.0,2.0,0.0
max,1.0,65.0,79000.0,6.0,5.0


# Преоразование матрицы признаков для защиты персональных данных

#### Какие действия необходимо сделать:

Доказательство подтверждает возможность умножения матрицы признаков на число. Также можно добавить StandartScaler(). Важно: просто Scaler (без предварительных преобразований) прменять не стоит, т.к. при больших выборках можно среднее по городу перенести на среднее по страховой компании и выявить дисперсию и среднее до Scaler для восстановления данных.

In [22]:
''' 
Напишем функцию трекнировки простейшей модели и проверим 
'''

def model_fit_predict_metric(X_train,
                             y_train,
                             X_test,
                             y_test):
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    r2_metric = r2_score(y_true = y_test,
                         y_pred = y_pred)

    print(f'r2 koef: {r2_metric.round(2)}')

#### Создание рандомного множителя и рандомного смещения

In [23]:
koef = np.random.normal(scale = 100)
bias = np.random.normal(scale = 1000)

print(f'KOEF: {koef}')
print(f'BIAS: {bias}')

KOEF: 141.28884544835705
BIAS: 372.1476737924873


#### Разбиение на тренировочную и тестовую выборку

In [24]:
X = data.iloc[:,:-1]
y = data.iloc[:,-1]

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    y,
                                                    test_size = 0.2,
                                                    random_state = RANDOM_STATE)

#### Преобразуем матрицу признаков

In [25]:
X_train_shift = X_train*koef + bias
X_test_shift = X_test*koef + bias

#### Проверка моделей не преобразованной матрицы признаков и с шифтом

In [26]:
print('Базовая модель без преобразований признаков \n')
model_fit_predict_metric(X_train,
                         y_train,
                         X_test,
                         y_test)
print('\n')
print('Модель со смещенными признаками \n')
model_fit_predict_metric(X_train_shift,
                         y_train,
                         X_test_shift,
                         y_test)

Базовая модель без преобразований признаков 

r2 koef: 0.44


Модель со смещенными признаками 

r2 koef: 0.44


# Вывод

- было математически доказано, что смещение матрицы признаков не повлияет на адекватность модели
- произвели преобразование матрицы признаков
- проверили первоначальную модель и модель с преобразованными признаками --> адекватность модели одинаковая